### Práctica de Web Scraping

Se va a considerar una página web de wikipedia donde se encuentra un recopilatorio de las pinturas realizadas por Velázquez: https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Vel%C3%A1zquez

En la página aparecen varios apartados, de los cuales se va a procesar aquel denominado "Obras total o parcialmente de Velázquez". En este apartado aparecen un conjunto de tablas clasificadas por diferentes etapas artísticas del pintor. En cada tabla aparecen pinturas junto a información asociada a cada una de ellas.

# Se pide hacer un buscador que permita recuperar información de la página. Para ello se le mostrará al usuario tres opciones entre las que tiene que elegir: Buscar por etapa pictórica, Buscar por técnica o Buscar por museo.

__Buscar por etapa pictórica[2,5 puntos]__
 
Si elige esta opción se le mostrará un listado de todas las etapas que aparecen en wikipedia antes mencionadas. Para facilitar la selección se asignará un número a cada etapa, de manera que el usuario elija por número.Las etapas pueden recuperarse de acuerdo a la caracterización antes indicada (bien usando las etiquetas h2 y h3 o bien por el listado que aparece al comienzo de la página). Una vez elegido una etapa, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas:  Título, Fecha, Técnica, Dimensiones, y observaciones. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre etapa pictórica.

__Buscar por técnica[2,5 puntos]__
 
Si elige esta opción se le mostrará un listado de todas las técnicas usadas por Velázques para pintar sus cuadros. Para facilitar la selección se asignará un número a cada técnica, de manera que el usuario elija por número.Las técnicas se pueden recuperar de la  columna Técnica de cada tabla de la página de wikipedia. Una vez elegida una técnica, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas:  Título, Fecha,  Dimensiones,Localización y observaciones. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre técnica.

__Buscar por museo[5 puntos]__
 
Si elige esta opción se le mostrará un listado de todos los museos en los que existe una pintura de Velázquez. Para facilitar la selección se asignará un número a cada museo, de manera que el usuario elija por número. Los museos se pueden recuperar de la  columna Localización de cada tabla de la página de wikipedia. Una vez elegido el museo, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas: Título, Fecha, Técnica, Dimensiones, y observaciones. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre museo.

__Nota: Para procesar la página html es obligatorio utilizar la libreria BeautifulSoup. No se pueden usar otras librerías no vistas en clase__

In [2]:
import requests
import urllib.request
import os
from bs4 import BeautifulSoup

url="https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Vel%C3%A1zquez"


def etapasPictoricas(url):
    
    listaEtapas = []
    r = requests.get(url)
    html = r.text
#*************************  Scrappeo para conseguir las etapas pictoricas  *************************

    soup1=BeautifulSoup(html,'html.parser')
    res = soup1.find_all("h3") #h3 es la etiqueta donde estan las etapas
    for re in res:   
        titulo = re.find("span", {"class" : "mw-headline"}) #analizando el esquema, el titulo esta en etiquetas span
        if titulo != None:
            listaEtapas.append(titulo.text)

#*************************  Tras conseguir las etapas, empieza el programa con un menu  *************************

    print("Elige la etapa pictorica: ")
    i = 0
    for titulo in listaEtapas:
        print(i + 1, titulo)
        i = i + 1
    elect = int(input()) #El usuario elige una etapa

    soup2=BeautifulSoup(html,'html.parser')
    
#Los datos estan agrupados en una etiqueta tabla, donde cada fila de la tabla se almacena en una etiqueta tr

    tabla = soup2.find_all("table") #Posiciones [0 - 5] pertenecen al apartado "Obras total o parcialmente de Velázquez"
    filas = tabla[elect - 1].find_all("tr")
    
    #Creacion del directorio con el nombre de la etapa elegida
    os.makedirs(listaEtapas[elect-1], exist_ok=True)
    i = 0 #Contador que sera el nombre de cada cuadro dentro del directorio
    for datos in filas[1:]:
        i = i + 1
        #cada dato del cuadro, esta almacenado en etiquetas td
        row = datos.find_all("td")
        urlImg = "https:" + row[0].find("img").get("src")
        img = urllib.request.urlopen(urlImg).read()
        manf=open( listaEtapas[elect-1] +"/"+ str(i) +".jpg","wb")
        manf.write(img)
        manf.close()
        #Uso de split ya que en cada dato hay \n, y lo queremos quitar
        print("Titulo: ", row[1].text.split("\n")[0])
        print("Fecha: ", row[2].text.split("\n")[0])
        print("Tecnica: ", row[3].text.split("\n")[0])
        print("Dimensiones: ", row[4].text.split("\n")[0])
        print("Observaciones: ", row[6].text.split("\n")[0])
        print()
        
        
def tecnicas(url):

    #*************************  Obtenición de técnicas  *************************
    r = requests.get(url)
    html = r.text
    
    tecnicas = {} #clave: Tecnica | valor: lista de diccionarios de las obras con dicha técnica
    listaTecnicas = [] #Lista que contendrá todas las técnicas del apartado "Obras total o parcialmente de Velázquez"
    
    #Cabecera donde contiene todas las etapas
    soup1=BeautifulSoup(html,'html.parser')
    tablas = soup1.find_all("table")
            
    for tabla in tablas: #Se ejecutará todas las etapas de las obras de Velázquez, ignorando el resto de apartados      
        #Los datos estan agrupados en una etiqueta tabla, donde cada fila de la tabla se almacena en una etiqueta tr
        filas = tabla.find_all("tr")
 
        for datos in filas[1:]:
            
            listaAuxCuadros = []
            subDict = {}
            row = datos.find_all("td")

            subDict["imagen"] = row[0].find("img").get("src")
            subDict["titulo"] = row[1].text.split("\n")[0]
            subDict["fecha"] = row[2].text.split("\n")[0]
            subDict["dimensiones"] = row[4].text.split("\n")[0]
            subDict["localizacion"] = row[5].text.split("\n")[0]
            subDict["observaciones"] = row[6].text.split("\n")[0]
            
            tecnica = row[3].text.split("\n")[0]
            #Si ya existe la tecnica, se añade en la lista de cuadros dentro del diccionario de tecnicas
            if tecnica in tecnicas:
                listaAuxCuadros = tecnicas[tecnica]
                listaAuxCuadros.append(subDict)
                tecnicas[tecnica] = listaAuxCuadros
            #Si no existe, se añade la nueva tecnica a la listaTecnicas y se empieza una nueva lista de cuadros dentro del diccionario de tecnicas
            else:
                #Comprobamos si la tecnica tiene un nombre o es vacío (ya que se dan casos en la tabla)
                if len(tecnica) != 0:
                    listaAuxCuadros.append(subDict)
                    tecnicas[tecnica] = listaAuxCuadros
                    listaTecnicas.append(tecnica)

#*************************  Tras conseguir las tecnicas, empieza el programa con un menu  *************************
    
    print("Elige una tecnica: ")
    i = 0
    for tecnica in listaTecnicas:
        print(i + 1, tecnica)
        i = i + 1
    elect = int(input())
    
    listaCuadros = [] #Lista que contendrá los cuadros con la ténica seleccionada
    listaCuadros = tecnicas[listaTecnicas[elect - 1]]
    
    #Creacion del directorio con el nombre de la tecnica elegida
    os.makedirs(listaTecnicas[elect-1], exist_ok=True)
    i = 0   
    for cuadro in listaCuadros:
        
        i = i + 1
        urlImg = "https:" + cuadro["imagen"]
        img = urllib.request.urlopen(urlImg).read()
        manf=open( listaTecnicas[elect-1] +"/"+ str(i) +".jpg","wb")
        manf.write(img)
        manf.close()
        
        print("Titulo: ", cuadro.get("titulo"))
        print("Fecha: ", cuadro.get("fecha"))
        print("Dimensiones: ", cuadro.get("dimensiones"))
        print("Localizacion: ", cuadro.get("localizacion"))
        print("Observaciones: ", cuadro.get("observaciones"))
        print()
        
def museos(url):
    
    #*************************  Obtenición de los museos  *************************
    #Mismo desarrollo que en el ejercicio 2
    
    r = requests.get(url)
    html = r.text
    museos = {}
    listaMuseos = []
    
    
    soup2=BeautifulSoup(html,'html.parser')
    tablas = soup2.find_all("table") 
    
    for tabla in tablas:

        filas = tabla.find_all("tr")

        for datos in filas[1:]:

            listaAuxCuadros = []
            subDict = {}
            row = datos.find_all("td")

            subDict["imagen"] = row[0].find("img").get("src")
            subDict["titulo"] = row[1].text.split("\n")[0]
            subDict["fecha"] = row[2].text.split("\n")[0]
            subDict["tecnica"] = row[3].text.split("\n")[0]
            subDict["dimensiones"] = row[4].text.split("\n")[0]
            subDict["observaciones"] = row[6].text.split("\n")[0]

            museo = row[5].text.split("\n")[0]
            if museo in museos:
                listaAuxCuadros = museos[museo]
                listaAuxCuadros.append(subDict)
                museos[museo] = listaAuxCuadros
            else:
                listaAuxCuadros.append(subDict)
                museos[museo] = listaAuxCuadros
                listaMuseos.append(museo)

 #*************************  Tras conseguir los museos, empieza el programa con un menu  *************************               
    print("Elige el museo: ")
    i = 0
    for museo in listaMuseos:
        print(i + 1, museo)
        i = i + 1
    elect = int(input())
    listaCuadros = []
    listaCuadros = museos[listaMuseos[elect - 1]]
    
    #Creacion del directorio con el nombre de la etapa elegida
    os.makedirs(listaMuseos[elect-1], exist_ok=True)
    i = 0 #Contador que sera el nombre de cada cuadro dentro del directorio
    for cuadro in listaCuadros:
        i = i + 1
        urlImg = "https:" + cuadro["imagen"]
        img = urllib.request.urlopen(urlImg).read()
        manf=open( listaMuseos[elect-1] +"/"+ str(i) +".jpg","wb")
        manf.write(img)
        manf.close()
        
        print("Titulo: ", cuadro.get("titulo"))
        print("Fecha: ", cuadro.get("fecha"))
        print("Tecnica: ", cuadro.get("tecnica"))
        print("Dimensiones: ", cuadro.get("dimensiones"))
        print("Observaciones: ", cuadro.get("observaciones"))
        print()
    
#*************************  Menu principal  ************************* 

opc = 10 #Valor auxiliar para iniciar el programa
while (opc != 0):
    print("Elige uno de los buscadores: ")
    print("1 Buscar por Etapas Pictoricas")
    print("2 Buscar por tecnica")
    print("3 Buscar por museo")
    print("0 Salir")
    opc = int(input())
    if opc == 1:
        etapasPictoricas(url)
    elif opc == 2:
        tecnicas(url)
    elif opc == 3:
        museos(url)

Elige uno de los buscadores: 
1 Buscar por Etapas Pictoricas
2 Buscar por tecnica
3 Buscar por museo
0 Salir
2
Elige una tecnica: 
1 Óleo sobre lienzo
2 Óleo sobre tabla
3 Óleo sobre cobre
4 Carboncillo sobre papel
5 Lápiz negro sobre papel
6 Lápiz negro sobre papel amarillento verjurado
7 Óleo sobre lienzo pegado a tabla
8 Óleo sobre tela
9 Aguafuerte
9
Titulo:  Francisco de Ochoa
Fecha:  
Dimensiones:  280 x 165 mm
Localizacion:  Biblioteca Nacional de España, Madrid
Observaciones:  Al retratado, Francisco de Ochoa, se le documenta como portero de cámara en 1665. En el palacio del Buen Retiro se inventarió en 1701 como de Velázquez un «Ochoa portero de corte con unos memoriales». En 1772, describiéndose como «Un retrato de un Alcalde con su vara y un legajo de memoriales», el cuadro se atribuía a Carreño. Grabado por Goya hacia 1780, una prueba del primer estado conservada en la Biblioteca Nacional de Madrid lleva un pie manuscrito que dice: «Quadro de Dn Diego Velázquez de Silva / G

# Normas de entrega

* Fecha tope de entrega: 12/11/2020
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre WebScraping_GrupoX donde X será el número de grupo correspondiente.
